# <h1><center>Implementações</center></h1>

## Leitura da Base de Dados

###  Importando as bibliotecas utilizadas na tarefa

In [ ]:
import pandas as pd

### Definindo a função que realizará a leitura da base de dados disponíveis no diretório

In [ ]:
def leitura(diretorio):
    serie = pd.read_csv(diretorio)
    return serie

## Pré-processamento de dados

In [ ]:
def removendo_valores_discrepantes(dados_historicos):
    dados_historicos = dados_historicos[dados_historicos['Volume']>0]
    return dados_historicos

In [ ]:
def removendo_dados_duplicados(dados_historicos):
    dados_historicos = dados_historicos.drop_duplicates()
    return dados_historicos

In [ ]:
def remover_coluna_date(dados_historicos):
    dados_sem_data = dados_historicos.drop(columns=['Date'])
    return dados_sem_data

## Gerando os indicadores técnicos

###  Importando as bibliotecas utilizadas na tarefa

In [ ]:
import talib as tb

In [ ]:
import operator

In [ ]:
import numpy as np

### Definindo a função que retornará os indicadores técnicos

#### Indicador baseado em médias móveis: suavização de preços anteriores

In [ ]:
def MMS(dados_historicos, periodo):
    
    # Média Móvel Simples:
    dados_historicos['MMS'] = tb.SMA(dados_historicos['Close'], timeperiod= periodo)

    ##Problema do Nan:
    dados_historicos['MMS'].values[:int(periodo)-1]= dados_historicos['MMS'].values[int(periodo)-1]
    
    ##Classificação:
    dif = dados_historicos['MMS'].values[1:]-dados_historicos['MMS'].values[:-1]
    dif = np.concatenate([[np.nan], dif])
    dados_historicos['DIF_MMS']= dif
    dados_historicos['ClasseMMS']= (dados_historicos['DIF_MMS']>0).astype(int)
    dados_historicos['ClasseMMS'].values[:1] = 0
    
    
    return dados_historicos

In [ ]:
def MME(dados_historicos, periodo):
    
    
    # Média Móvel Exponencial:
    dados_historicos['MME'] = tb.EMA(dados_historicos['Close'], timeperiod= periodo)

    ##Problema do Nan:
    dados_historicos['MME'].values[:int(periodo)-1]= dados_historicos['MME'].values[int(periodo)-1]
    
    ##Classificação:
    
    dif_1 = dados_historicos['MME'].values[1:]-dados_historicos['MME'].values[:-1]
    dif_1 = np.concatenate([[np.nan], dif_1])
    dados_historicos['DIF_MME']= dif_1
    dados_historicos['ClasseMME']= (dados_historicos['DIF_MME']>0).astype(int)
    dados_historicos['ClasseMME'].values[:1] = 0
    return dados_historicos

In [2]:
def MACD (dados_historicos):
    # MACD:
    dados_historicos['MACD'], dados_historicos['sinal_MACD'], dados_historicos['macdhist'] = tb.MACD(dados_historicos['Close'], fastperiod=12, slowperiod=26, signalperiod=9)
    ##Problema do Nan:
    dados_historicos['MACD'].values[:33]= dados_historicos['MACD'].values[33]
    dados_historicos['sinal_MACD'].values[:33]= dados_historicos['sinal_MACD'].values[33]
    dados_historicos['macdhist'].values[:33]= dados_historicos['macdhist'].values[33]
    ##Classificando o macd:
    dados_historicos['ClasseMACD']=0
    dados_historicos.loc[dados_historicos['sinal_MACD']<dados_historicos['MACD'],'ClasseMACD']=1
    return dados_historicos

In [3]:
def RSI(dados_historicos, periodo):
    dados_historicos['RSI'] = tb.RSI(dados_historicos['Close'] , periodo)
    
    #Problema do Nan:
    dados_historicos['RSI'].values[:periodo]= dados_historicos['RSI'].values[periodo]
    
    #Classificação:
    dados_historicos['ClasseRSI']=1
    dados_historicos.loc[dados_historicos['RSI']<50,'ClasseRSI']=0
    
    return dados_historicos

## Definindo os conjuntos de treinamento e teste

### Somente indicadores técnicos

In [ ]:
def  conjunto_indicadores_treinamento(dados_historicos):
    classe = pd.DataFrame()
    classe = pd.DataFrame()
    classe['MMS'] = dados_historicos['MMS'].values[:int(len(dados_historicos)*0.7)+1]
    classe['MME'] = dados_historicos['MME'].values[:int(len(dados_historicos)*0.7)+1]
    classe['MACD'] = dados_historicos['MACD'].values[:int(len(dados_historicos)*0.7)+1]
    classe['RSI'] = dados_historicos['RSI'].values[:int(len(dados_historicos)*0.7)+1]
 
    return classe

In [ ]:
def  conjunto_indicadores_teste(dados_historicos):
    classe_teste = pd.DataFrame()
    classe_teste['MMS'] = dados_historicos['MMS'].values[1+int(len(dados_historicos)*0.7):]
    classe_teste['MME'] = dados_historicos['MME'].values[1+int(len(dados_historicos)*0.7):]
    classe_teste['MACD'] = dados_historicos['MACD'].values[1+int(len(dados_historicos)*0.7):]
    classe_teste['RSI'] = dados_historicos['RSI'].values[1+int(len(dados_historicos)*0.7):]
    return classe_teste

## Rotulação das amostras

In [ ]:
def target_treinamento(dados_historicos):
    soma = dados_historicos['ClasseMMS']+dados_historicos['ClasseMME']+dados_historicos['ClasseMACD']+dados_historicos['ClasseRSI']
    dados_historicos['soma'] = soma
    dados_historicos['Soma'] = (dados_historicos['soma']>2).astype(int)
    classe = pd.DataFrame()
    classe['target'] = dados_historicos['Soma'].values[:int(len(dados_historicos)*0.7)+1]
    return classe

In [ ]:
def target_teste(dados_historicos):
    soma = dados_historicos['ClasseMMS']+dados_historicos['ClasseMME']+dados_historicos['ClasseMACD']+dados_historicos['ClasseRSI']
    dados_historicos['soma'] = soma
    dados_historicos['Soma'] = (dados_historicos['soma']>2).astype(int)
    classe = pd.DataFrame()
    classe['target'] = dados_historicos['Soma'].values[int(len(dados_historicos)*0.7)+1:]
    return classe

## Normalizando a base de dados

###  Importando as bibliotecas utilizadas na tarefa

In [ ]:
from sklearn.preprocessing import MinMaxScaler

###  Definindo a função que normalizará  a base de dados

In [ ]:
def normalizador(indicadores):
    indicadores = MinMaxScaler(feature_range=(0,1)).fit_transform(indicadores)
    return indicadores

## Adequando o formato do conjunto de rótulos

###  Importando as bibliotecas utilizadas na tarefa

In [ ]:
import numpy as np

### Definindo a função responsável por redimensionar o conjunto

In [ ]:
def reshape_target(target):
    target_reshape = target.reshape(len(target), )
    return target_reshape

## Avaliando os modelos criados na sequência

### Importando as bibliotecas utilizadas na tarefa

In [2]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report
from sklearn.metrics import recall_score, f1_score, precision_score
from sklearn.metrics import roc_curve, plot_roc_curve, roc_auc_score
import matplotlib.pyplot as plt

### Definindo a  função responsável por executar a tarefa

In [6]:
def relatorio(previsao_do_modelo, target, proba, nome_fig, nome_mtz):
    matriz_confusão = confusion_matrix(target, previsao_do_modelo)
    print(f'Falsos positivo - fp: {matriz_confusão[0,1]}')
    print(f'Verdadeiros positivo - tp: {matriz_confusão[1,1]}')
    print(f'Falsos negativo - fn: {matriz_confusão[1,0]}')
    print(f'Verdadeiros negativo  - tn: {matriz_confusão[0,0]}')
    print(f'Recall: {round(recall_score(target, previsao_do_modelo),2)}')
    print(f'F1-score: {round(f1_score(target, previsao_do_modelo),2)}')
    print(f'Precisão: {round(precision_score(target, previsao_do_modelo), 2)}')
    print(f'Acurácia: {round(accuracy_score(target, previsao_do_modelo),2)}')
    
    print('---------------------------------------------')
    
    figura, eixo = plt.subplots(figsize=(6, 6))
    eixo.matshow(matriz_confusão, cmap=plt.cm.Greens)
    for i in range(matriz_confusão.shape[0]):
        for j in range(matriz_confusão.shape[1]):
            eixo.text(x=j, y=i,s=matriz_confusão[i, j], va='center', ha='center', size='xx-large')
 
    plt.xlabel('Predição', fontsize=15)
    plt.ylabel('Real', fontsize=15)
    plt.title('Matriz de Confusão', fontsize=18)
    plt.savefig(nome_mtz, dpi = 300)
    
    
    print('---------------------------------------------')
    
    #taxa_fp, taxa_vp, limiar = roc_curve(target, previsao_do_modelo)
    taxa_fp, taxa_vp, limiar = roc_curve(target, proba[:,1])
    auc = roc_auc_score(target, proba[:,1])
    

    figure = plt.figure(figsize = (6, 6))
    plt.plot(taxa_fp, taxa_vp, linewidth = 2, label='AUC = %0.2f' % auc)
    plt.title('Curva ROC')
    plt.plot([0, 1], [0, 1],'k--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('Taxa de verdadeiro positivo')
    plt.xlabel('Taxa de falso positivo ')
    plt.legend(loc="lower right")
    plt.savefig(nome_fig, dpi = 300)
    
    return figura, figure
    
    

In [7]:
def relatorio_lstm(previsao_do_modelo, target, proba, nome_fig, nome_mtz):
    matriz_confusão = confusion_matrix(target, previsao_do_modelo)
    print(f'Falsos positivo - fp: {matriz_confusão[0,1]}')
    print(f'Verdadeiros positivo - tp: {matriz_confusão[1,1]}')
    print(f'Falsos negativo - fn: {matriz_confusão[1,0]}')
    print(f'Verdadeiros negativo  - tn: {matriz_confusão[0,0]}')
    print(f'Recall: {round(recall_score(target, previsao_do_modelo),2)}')
    print(f'F1-score: {round(f1_score(target, previsao_do_modelo),2)}')
    print(f'Precisão: {round(precision_score(target, previsao_do_modelo), 2)}')
    print(f'Acurácia: {round(accuracy_score(target, previsao_do_modelo),2)}')
    
    print('---------------------------------------------')
    
    figura, eixo = plt.subplots(figsize=(6, 6))
    eixo.matshow(matriz_confusão, cmap=plt.cm.Greens)
    for i in range(matriz_confusão.shape[0]):
        for j in range(matriz_confusão.shape[1]):
            eixo.text(x=j, y=i,s=matriz_confusão[i, j], va='center', ha='center', size='xx-large')
 
    plt.xlabel('Predição', fontsize=15)
    plt.ylabel('Real', fontsize=15)
    plt.title('Matriz de Confusão', fontsize=18)
    plt.savefig(nome_mtz, dpi = 300)
    
    
    print('---------------------------------------------')
    
    #taxa_fp, taxa_vp, limiar = roc_curve(target, previsao_do_modelo)
    taxa_fp, taxa_vp, limiar = roc_curve(target, proba[:])
    auc = roc_auc_score(target, proba[:])
    
    
    figure = plt.figure(figsize = (6, 6))
    plt.plot(taxa_fp, taxa_vp, linewidth = 2, label='AUC = %0.2f' % auc)
    plt.title('Curva ROC')
    plt.plot([0, 1], [0, 1],'k--')
    plt.xlim([0, 1])
    plt.ylim([0, 1])
    plt.ylabel('Taxa de verdadeiro positivo')
    plt.xlabel('Taxa de falso positivo ')
    plt.legend(loc="lower right")
    plt.savefig(nome_fig, dpi = 300)
    
    return figura, figure
    
    

In [ ]:
def grafico(x, y):
    grafico = plt.plot(x, y)
    return grafico

## SVM

###  Importando as bibliotecas utilizadas na tarefa

In [ ]:
# SVM:
from sklearn.svm import SVC

### Definindo a função que criará o modelo

In [ ]:
def modelo_SVM(C, kernel, gamma, vetor_caracteristica_treinamento, rotulo_treinamento, vetor_caracteristica_teste):
    model = SVC(C = C, kernel= kernel, gamma= gamma, probability=True) 
    model.fit(vetor_caracteristica_treinamento, rotulo_treinamento)
    predicted= model.predict(vetor_caracteristica_teste)
    return predicted, model.predict_proba(vetor_caracteristica_teste)

### Utilizando o GridSearchCV

#### Importando as bibliotecas utilizadas na tarefa

In [ ]:
from sklearn.model_selection import GridSearchCV

In [ ]:
def model_grid(x_treinamento, x_teste, y_treinamento, y_teste):
    # Definindo os parâmetros:
    parametros_grid = {'C':  [0.1,1, 10, 100], 
              'gamma': [1, 0.1, 0.01, 0.001, 0.0001],
              'kernel': ['rbf', 'poly', 'linear', 'sigmoid']} 
    # Criando o modelo em que a grade de parâmetros será testada:
    grid = GridSearchCV(SVC(probability = True), parametros_grid, refit = True, verbose = 0)
    # Treinando o modelo:
    grid.fit(x_treinamento, y_treinamento)
    # Imprimindo os melhores parâmetros:
    print('Melhores parâmetros')
    print(grid.best_params_)
    # Imprimindo as configurações do melhor estimador:
    print('Melhor estimador')
    print(grid.best_estimator_)
    # Realizando as predições:
    grid_predictions = grid.predict(x_teste)
    
    print('---------------------------------------------')
    
    # Imprimindo as métricas de avaliação:
    print(classification_report(y_teste, grid_predictions))
    
    print('---------------------------------------------')
    
 
    relatorio(grid_predictions, y_teste)

## LSTM

### Importando as bibliotecas utilizadas na tarefa

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, LSTM, Dropout
import time

### Definindo as funções responsáveis por executar a tarefa

#### Modificando a entrada para o formato exigido pela camada 'LSTM' do Keras

In [ ]:
def modificar_formato_entrada(indicadores):
    # Arrumando a entrada do conjunto de treinamento no formato exigido pelo LSTM
    x = indicadores
    x = x.reshape((x.shape[0],x.shape[1],1))
    return x

#### Definindo os modelos

In [ ]:
def create_model(xtreino_novo):
    model = Sequential()
    model.add(LSTM(50, activation= 'relu', input_shape = (xtreino_novo.shape[1],xtreino_novo.shape[2])))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [10]:
def lstm1(indicadores_treinamento, indicadores_teste, target_treinamento, target_teste):
    
    startTime = time.time()
    
    # Arrumando a entrada dos conjuntos de treinamento e teste no formato exigido pelo LSTM
    xtreino_novo = modificar_formato_entrada(indicadores_treinamento)
    xteste_novo = modificar_formato_entrada(indicadores_teste)
    
    # Modelo 1:
    model = create_model(xtreino_novo)

    # Resumo da Rede Neural:
    print('Resumo da Rede Neural:')
    model.summary()
    print('---------------------------------------------')
    
    # Ajuste da Rede Neural ao conjunto de treinamento:
    model.fit(xtreino_novo, target_treinamento, epochs = 10, batch_size = 1)
    executionTime = (time.time() - startTime)
    print('Tempo de Execução: ' + str(executionTime))
    
    # Ajustando o modelo ao conjunto de entrada de teste: 
    y_predict = model.predict_classes(xteste_novo)
    
    return y_predict, model.predict_proba(xteste_novo)

In [37]:
def create_model2(xtreino_novo):
    model = Sequential()
    model.add(LSTM(units=50, activation='relu', return_sequences=True, input_shape=(xtreino_novo.shape[1],xtreino_novo.shape[2])))
    model.add(Dropout(0.2))
    model.add(LSTM(units=50, activation='relu', return_sequences=False))
    model.add(Dropout(0.2))
    model.add(Dense(1, activation='sigmoid'))

    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [42]:
def lstm2(indicadores_treinamento, indicadores_teste, target_treinamento, target_teste):
    
    
    startTime = time.time()
    
    # Arrumando a entrada dos conjuntos de treinamento e teste no formato exigido pelo LSTM
    xtreino_novo = modificar_formato_entrada(indicadores_treinamento)
    xteste_novo = modificar_formato_entrada(indicadores_teste)
    
    # Modelo 2:
    model = create_model2(xtreino_novo)

    # Resumo da Rede Neural:
    print('Resumo da Rede Neural:')
    model.summary()
    print('---------------------------------------------')
    
    # Ajuste da Rede Neural ao conjunto de treinamento:
    model.fit(xtreino_novo, target_treinamento, batch_size=64, epochs=170)
    executionTime = (time.time() - startTime)
    print('Tempo de Execução: ' + str(executionTime))
    
    # Ajustando o modelo ao conjunto de entrada de teste: 
    y_predict = model.predict_classes(xteste_novo)
    
    return y_predict, model.predict_proba(xteste_novo)

In [39]:
def create_model3(xtreino_novo):
    model = Sequential()
    model.add(LSTM(12, activation='relu', return_sequences = True, input_shape = (xtreino_novo.shape[1],xtreino_novo.shape[2])))
    model.add(LSTM(8, activation='relu'))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [11]:
def lstm3(indicadores_treinamento, indicadores_teste, target_treinamento, target_teste):
    
    
    startTime = time.time()
    
    # Arrumando a entrada dos conjuntos de treinamento e teste no formato exigido pelo LSTM
    xtreino_novo = modificar_formato_entrada(indicadores_treinamento)
    xteste_novo = modificar_formato_entrada(indicadores_teste)
    
    # Modelo 3:
    model = create_model3(xtreino_novo)

    # Resumo da Rede Neural:
    print('Resumo da Rede Neural:')
    model.summary()
    print('---------------------------------------------')
    
    # Ajuste da Rede Neural ao conjunto de treinamento:
    model.fit(xtreino_novo, target_treinamento, epochs = 10, batch_size = 1)
    executionTime = (time.time() - startTime)
    print('Tempo de Execução: ' + str(executionTime))
    
    # Ajustando o modelo ao conjunto de entrada de teste: 
    y_predict = model.predict_classes(xteste_novo)
    
    return y_predict, model.predict_proba(xteste_novo)

In [12]:
def create_model4(xtreino_novo):
    model = Sequential()
    model.add(LSTM(30, activation= 'relu', input_shape = (xtreino_novo.shape[1],xtreino_novo.shape[2])))
    model.add(Dense(units=1, activation='sigmoid'))
    model.compile(optimizer='rmsprop', loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [13]:
def lstm4(indicadores_treinamento, indicadores_teste, target_treinamento, target_teste):
    
    startTime = time.time()
    
    # Arrumando a entrada dos conjuntos de treinamento e teste no formato exigido pelo LSTM
    xtreino_novo = modificar_formato_entrada(indicadores_treinamento)
    xteste_novo = modificar_formato_entrada(indicadores_teste)
    
    # Modelo 4:
    model = create_model4(xtreino_novo)

    # Resumo da Rede Neural:
    print('Resumo da Rede Neural:')
    model.summary()
    print('---------------------------------------------')
    
    # Ajuste da Rede Neural ao conjunto de treinamento:
    model.fit(xtreino_novo, target_treinamento, epochs = 10, batch_size = 1)
    executionTime = (time.time() - startTime)
    print('Tempo de Execução: ' + str(executionTime))
    
    # Ajustando o modelo ao conjunto de entrada de teste: 
    y_predict = model.predict_classes(xteste_novo)
    return y_predict, model.predict_proba(xteste_novo)